In [ ]:
from googleapiclient.discovery import build
DEVELOPER_KEY = open('api_key.txt').read()
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)


In [ ]:
def getIDfromName(name):
    request = youtube.search().list(
        part="snippet",
        q=name,
        type="channel",
        maxResults=5
    )
    response = request.execute()
    return response['items'][0]['id']['channelId']
name = "Barbascura eXtra"
channel_id = getIDfromName(name)
channel_id

In [ ]:
def getChannelPlaylist(channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails",
        id=channel_id
    )
    response = request.execute()
    return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
playlist_id = getChannelPlaylist(channel_id)
playlist_id

In [ ]:
def getAllVideoFromPlaylist(playlist_id):
    video_ids = []
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults=50
    )
    response = request.execute()
    video_ids = [item['snippet']['resourceId']['videoId'] for item in response['items']]
    nextPageToken = response.get('nextPageToken', None)
    while nextPageToken:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=nextPageToken
        )
        response = request.execute()
        for item in response['items']:
            video_ids.append(item['snippet']['resourceId']['videoId'])
            
        nextPageToken = request.execute().get('nextPageToken', None)
    return video_ids
video_ids = getAllVideoFromPlaylist(playlist_id)
print(len(video_ids))

In [ ]:
def getCommentatorsForVideo(video_id,number):
    commentators, nextPageToken = OnePageCommentators(video_id,number)
    
    while nextPageToken:
        newComments = []
        newComments, nextPageToken = OnePageCommentators(video_id, nextPageToken)
        commentators.extend(newComments)
    return commentators

def OnePageCommentators(video_id, number, nextPageToken=None):
    commentators = []
    if nextPageToken:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            maxResults=number,
            pageToken=nextPageToken
        )
    else:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            maxResults=number
        )
    
    response = request.execute()
    # print("Response items: ", len(response['items']))
    for item in response['items']:
        commentators.append(item['snippet']['topLevelComment']['snippet']['authorDisplayName'])
        if 'replies' in item:
            # print("Replies", len(item['replies']['comments']))
            for reply in item['replies']['comments']:
                commentators.append(reply['snippet']['authorDisplayName'])
    
    nextPageToken = response.get('nextPageToken', None)
    
    return commentators, nextPageToken

def getCommentatorsForAllVideos(video_ids,number):
    commentators = dict()
    sum = 0
    for video_id in video_ids:
        try:
            commentators[video_id] = getCommentatorsForVideo(video_id,number)
            # print(video_id, len(commentators[video_id]))
            sum += len(commentators[video_id])
        except Exception as e:
            print(e)
            continue
    print(sum)
    return commentators

commentators = getCommentatorsForAllVideos(video_ids,50)


In [ ]:
import networkx as nx
from itertools import combinations

def getSameCommentatorsNumber(video1_id, video2_id):
    return len(set(commentators[video1_id]).intersection(set(commentators[video2_id])))

def getGraph(commentators):
    maxCommentators = 0
    G = nx.Graph()
    for video_id in commentators.keys():
        G.add_node(video_id)
    for video1_id, video2_id in combinations(commentators.keys(), 2):
        if video1_id == video2_id:
            continue
        sameCommentators = getSameCommentatorsNumber(video1_id, video2_id)
        if sameCommentators > 2:
            G.add_edge(video1_id, video2_id, weight=sameCommentators)
    return G
#export graph in gexf format
G = getGraph(commentators)

def formatName(name):
    return name.replace(" ", "_").replace(".", "")
nx.write_gexf(G, formatName(name)+".gexf")
